## Import the Libraries

In [ ]:
import pyspark
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import StandardScaler
spark = SparkSession.builder.appName("Decision Trees").getOrCreate()

##  Download the Dataset 

In [2]:
!wget https://raw.githubusercontent.com/mananparasher/Spark-Datasets/master/bank_data.csv

--2020-06-20 23:46:00--  https://raw.githubusercontent.com/mananparasher/Spark-Datasets/master/bank_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.124.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.124.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 918960 (897K) [text/plain]
Saving to: ‘bank_data.csv.1’

bank_data.csv.1     100%[===================>] 897.42K  1.57MB/s    in 0.6s    

2020-06-20 23:46:00 (1.57 MB/s) - ‘bank_data.csv.1’ saved [918960/918960]



## Load the Data in Spark DataFrame 

In [2]:
df = spark.read.csv('bank_data.csv', header = True, inferSchema = True)
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)



## Data Processing for machine learning model

In [3]:
pipeline_stages=[]
categorical_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome']
for value in categorical_columns:
    string_indexer = StringIndexer(inputCol=value, outputCol=value+"indexer")
    pipeline_stages += [string_indexer]
    
target_column = StringIndexer(inputCol = 'deposit', outputCol = 'label')
pipeline_stages += [target_column]

numerical_columns = ['age', 'balance', 'duration', 'campaign', 'pdays', 'previous']
combined_columns = [columns + "indexer" for columns in categorical_columns] + numerical_columns

vector_assembler = VectorAssembler(inputCols=combined_columns, outputCol="features")
pipeline_stages += [vector_assembler]

In [4]:
pipeline_stages

[StringIndexer_40bba4c936b1022162d0,
 StringIndexer_4a2caee5e7956931d57b,
 StringIndexer_461db0ca38fdcdff3f66,
 StringIndexer_4f1fa47cfd0326f9efd6,
 StringIndexer_4c6da5d63db3957137a9,
 StringIndexer_43ffa19dedba98b240a4,
 StringIndexer_4b75ae5beeeb8b10c9da,
 StringIndexer_4ec184b85ef7526b3cea,
 StringIndexer_417f98d5db9bfea78251,
 VectorAssembler_4d7ca2a5ab98b8da2930]

In [5]:
pipeline = Pipeline(stages = pipeline_stages)
pipeline_fit = pipeline.fit(df).transform(df)
df = pipeline_fit.select(["features","label"])
df.printSchema()

root
 |-- features: vector (nullable = true)
 |-- label: double (nullable = false)



##  Data Splitting 

In [6]:
training_data, testing_data = df.randomSplit([0.9, 0.1])
print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(testing_data.count()))

Training Dataset Count: 10042
Test Dataset Count: 1120


## Model Implementation and Fitting 

In [9]:
decisiontreeclassifier = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label', maxDepth = 10)
model = decisiontreeclassifier.fit(training_data)
model

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_43ed9454cf0a5560ef66) of depth 10 with 861 nodes

## Model Prediction  

In [8]:
predictions = model.transform(testing_data)
predictions.select("features","prediction","label").show(10)

+--------------------+----------+-----+
|            features|prediction|label|
+--------------------+----------+-----+
|(14,[0,1,2,4,8,10...|       0.0|  1.0|
|(14,[0,1,2,4,8,10...|       0.0|  0.0|
|(14,[0,1,2,6,8,10...|       0.0|  0.0|
|(14,[0,1,2,6,8,10...|       1.0|  0.0|
|(14,[0,1,2,6,8,10...|       0.0|  0.0|
|(14,[0,1,2,8,9,10...|       0.0|  1.0|
|(14,[0,1,2,8,9,10...|       1.0|  1.0|
|(14,[0,1,2,8,9,10...|       0.0|  0.0|
|(14,[0,1,2,8,9,10...|       0.0|  0.0|
|(14,[0,1,2,8,9,10...|       1.0|  1.0|
+--------------------+----------+-----+
only showing top 10 rows

